**Table of contents**<a id='toc0_'></a>    
- [dynamic window approach(DWA)によるローカルパスプランニング](#toc1_)    
  - [参考サイト](#toc1_1_)    
- [pybulletの起動](#toc2_)    
- [LiDARクラスの定義](#toc3_)    
- [Dynamic Window Approachの定義](#toc4_)    
- [初期設定](#toc5_)    
- [移動ロボットのパラメータ設定](#toc6_)    
- [lidarのパラメータ設定](#toc7_)    
- [障害物設定](#toc8_)    
- [dynamic window approach パラメータ設定](#toc9_)    
- [dynamic window approachの実行](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[dynamic window approach(DWA)によるローカルパスプランニング](#toc0_)

本notebookではdynamic window approachについて説明します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）


dynamic window approachは、「ローカルパスプランニング」の手法の一つで、ロボットが目標地点に到達するために、障害物を避けながら進むための経路を計画する手法の1つです。

基本的には、ローカルパスプランニングのみでプランニングすることは少なく、A*のような「グローバルパスプランニング」と組み合わせて使用されることが多いです。

<br>

具体的には、

1. グローバルパスプランニングで初期地点から目標地点までの大域的な経路を計画します。
   
<img src="../images/MobileRobot/mobile_robot_local_path_planning_dwa/global_path_planning.png" width="50%">  

<br>
<br>

2. グローバルパスプランニングにより計画された大域的な経路をいくつかの中間地点に分割し、ローカルパスプランニングにより「中間地点1」→「中間地点2」→「中間地点3」→...→「目標地点」のように順番に走行します。


<img src="../images/MobileRobot/mobile_robot_local_path_planning_dwa/local_path_planning.png" width="50%">

なお、ローカルパスプランニングの場合は、リアルタイムに周りの環境を考慮しながら目標地点に走行します。  
そのため、障害物が少し移動する程度なら、グローバルパスプランニングを再計画することなく、障害物を避けながら目標地点に到達することができます。

## <a id='toc1_1_'></a>[参考サイト](#toc0_)
- [The Dynamic Window Approach to Collision Avoidance](https://www.ri.cmu.edu/pub_files/pub1/fox_dieter_1997_1/fox_dieter_1997_1.pdf)
- [Dynamic Window Approachによるローカルパスプランニングの基礎と強化学習との融合による移動障害物回避](https://developers.agirobots.com/jp/dynamic-window-approach/)
- [ロボットプログラミング 7 イントロ](https://www.robot.soc.i.kyoto-u.ac.jp/roboprog/v7_intro.html)
- [DWA（Dynamic Window Approach）についてまとめてみた](https://qiita.com/MENDY/items/16343a00d37d14234437)
- [PythonRobotics: dynamic_window_approach.py](https://github.com/AtsushiSakai/PythonRobotics/blob/master/PathPlanning/DynamicWindowApproach/dynamic_window_approach.py)
- [dwa_planner: dwa_planner.cpp](https://github.com/amslabtech/dwa_planner/blob/master/src/dwa_planner.cpp)

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

startThreads creating 1 threads.
starting thread 0


pybullet build time: Nov 28 2023 23:45:17


started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[LiDARクラスの定義](#toc0_)

In [2]:
import math 
import numpy as np

class Lidar():
    def __init__(self):
        self.replace_item_uniqueIds = None

    # 姿勢を表す回転行列
    def rotation_matrix(self, roll, pitch, yaw):
        R_x = [[1, 0, 0],
                [0, math.cos(roll), -math.sin(roll)],
                [0, math.sin(roll), math.cos(roll)]]
        
        R_y = [[math.cos(pitch), 0, math.sin(pitch)],
                [0, 1, 0],
                [-math.sin(pitch), 0, math.cos(pitch)]]
        
        R_z = [[math.cos(yaw), -math.sin(yaw), 0],
                [math.sin(yaw), math.cos(yaw), 0],
                [0, 0, 1]]
        
        return np.dot(R_z, np.dot(R_y, R_x))

    def CheckHits(self, ray_from_position, ray_orientation, ray_length, angle_resolution_rad, ray_direction_range_rad, offset, draw_debug_line=False):
        """
        姿勢を反映した障害物検知のためのレイキャスト
        Parameters
        ---
        ray_from_position : list
            rayが始まる位置を格納したlist [x, y, z]
        orientation : list
            レイ発射元の姿勢 [roll, pitch, yaw]
        ray_length : float
            rayの長さ
        angle_resolution_rad : float
            角度の分解能
        ray_direction_range_rad : float
            rayを発射する範囲を格納したlist [startRad, endRad]
        offset : float
            rayFromPositionからのオフセット
        draw_debug_line : bool
            rayを描画するかどうか
        
        Returns
        ---
        hit_results: tuple
            レイキャストの結果
        """
        
        
        R = self.rotation_matrix(ray_orientation[0], ray_orientation[1], ray_orientation[2])
        
        # 分割数
        num_divisions = int((ray_direction_range_rad[1] - ray_direction_range_rad[0]) / angle_resolution_rad)

        # ray_direction_range_radの範囲内で、numDivisions分だけrayの始点/終点を生成
        ray_froms = []
        ray_toes = []
        for i in range(num_divisions):
            angle = ray_direction_range_rad[0] + i * angle_resolution_rad
            direction = [math.cos(angle), math.sin(angle), 0]
            
            # 回転行列を適用して方向ベクトルを回転
            rotated_direction = np.dot(R, direction)

            # rayの始点
            ray_from = ray_from_position + offset * rotated_direction
            ray_froms.append(ray_from)

            # rayの終点
            ray_to = ray_from + ray_length * rotated_direction
            ray_toes.append(ray_to)
        
        # numDivisions分のレイキャストを実施
        hit_results = pybullet.rayTestBatch(ray_froms, ray_toes)

        # rayの描画
        if draw_debug_line:
            if self.replace_item_uniqueIds is None:
                self.replace_item_uniqueIds = []
                
                for i in range(num_divisions):
                    isHit = hit_results[i][0]
                    ray_from = ray_froms[i]
                    
                    if isHit != -1:
                        rayHitPosition = hit_results[i][3]
                        self.replace_item_uniqueIds.append(pybullet.addUserDebugLine(ray_from, rayHitPosition, lineColorRGB=[1,0,0]))
                    else:
                        rayToPosition = ray_toes[i]
                        self.replace_item_uniqueIds.append(pybullet.addUserDebugLine(ray_from, rayToPosition, lineColorRGB=[0,1,0]))
            
            else:
                for i in range(num_divisions):
                    isHit = hit_results[i][0]
                    ray_from = ray_froms[i]
                    
                    if isHit != -1:
                        rayHitPosition = hit_results[i][3]
                        pybullet.addUserDebugLine(ray_from, rayHitPosition, lineColorRGB=[1,0,0], replaceItemUniqueId=self.replace_item_uniqueIds[i])
                    else:
                        rayToPosition = ray_toes[i]
                        pybullet.addUserDebugLine(ray_from, rayToPosition, lineColorRGB=[0,1,0], replaceItemUniqueId=self.replace_item_uniqueIds[i])
        return hit_results


ven = Mesa


# <a id='toc4_'></a>[Dynamic Window Approachの定義](#toc0_)

Dynamic Windows Approach（DWA）は、ロボットの速度と角速度の組み合わせを評価し、最適な速度と角速度を選択する手法です。

おおまかな手順は以下の通りです。
1. 初期化
   - `__init__`メソッドで初期化を行います。
2. Vr (Velocity Result)の計算
   - `calc_vr`メソッドでVrを計算します。
   - VrはVs(Velocity Sample)、Vd(Velocity dynamic window),Va(Velocity admissible)の積集合になります。
     - Vs: ロボットの速度と角速度の範囲（ロボットの物理的なスペックにより決定される）
     - Vd: ロボットの現在速度・角速度から「微小時間dtの間に取りうる速度・角速度の範囲」
     - Va: ロボットが現在位置である速度、角速度で動作した際に、障害物と衝突しない速度、角速度の範囲
   <img src="../images/MobileRobot/mobile_robot_local_path_planning_dwa/dynamic_window_approach_overview.png" width="50%">
3. 最適な速度と角速度の選択
   - `calc_best_u`メソッドで Vr の範囲の中から、最適（最もいい感じに進んでくれそうな）な速度と角速度を選択します。
     1. 予測軌道の計算
          - `calc_predicted_trajectory`メソッドで、速度と角速度を元に`predict_time`後までの予測軌道を計算します。
     2. コストの計算：予測軌道に対して、以下のコストを計算します。
          - `calc_to_goal_cost`メソッド：軌道の終点がゴール方向を向いているか
          - `calc_speed_cost`メソッド：その軌道で進んだ時に、早い速度・角速度で進めるか
          - `calc_obstacle_cost`メソッド：その軌道で進んだ時に、障害物との距離が近くないか
     3. 上記の手順で計算した「最もコストが低くなる軌道になる速度と角速度」を制御入力として選択します。
4. ゴールに到達するまで、2.3を繰り返します。
 

In [ ]:
class Vs:
    """
    Vs(velocity space)の定義。
    ロボットがスペック的に取りうる速度と角速度の範囲を定義（移動ロボットのスペックに依存するため、基本的に範囲は固定）
    """
    def __init__(self, linear_vel_min, linear_vel_max, angle_vel_min, angle_vel_max):
        self.linear_vel_min = linear_vel_min
        self.linear_vel_max = linear_vel_max
        self.angle_vel_min = angle_vel_min
        self.angle_vel_max = angle_vel_max

class Vd:
    """
    Vd(Velocity dynamic window)の定義。
    ロボットの現在の速度・角速度から、「微小時刻dtの間に取りうる速度・角速度の範囲」を定義
    """
    def __init__(self):
        self.linear_vel_min = 0.0
        self.linear_vel_max = 0.0
        self.angle_vel_min = 0.0
        self.angle_vel_max = 0.0

    def calc_Vd(self, linear_vel_current, angle_vel_current, linear_acc_min, linear_acc_max, angle_acc_min, angle_acc_max, dt):
        self.linear_vel_min = linear_vel_current + linear_acc_min * dt
        self.linear_vel_max = linear_vel_current + linear_acc_max * dt
        self.angle_vel_min = angle_vel_current + angle_acc_min * dt
        self.angle_vel_max = angle_vel_current + angle_acc_max * dt

class Vr:
    """
    Vr(Velocity result)の定義。
    Vs,Vdの積集合（Vs,Vaの重なる範囲）を計算。
    論文上は、Vrは「Vs, Vd, Vaの積集合」として定義されているが、Va全体の領域を計算することは計算コストが高いため、ここではVs,Vdの積集合として定義する。
    Vaは、「calc_obstacle_cost」で「候補軌道上でのみ」領域判定を行っている（=候補軌道上がVa領域であるかどうかさえ分かればよいため、あらかじめVa全体を計算する必要はない）
    ※ Va(velocity admissible)：ロボットが現在位置で、ある速度・角速度で動作したときに「障害物と衝突しない速度・角速度の範囲」
       これは、Vs,Vdとは異なり、領域を求めるために衝突判定が必要なため、計算コストが高い
    """
    def __init__(self):
        self.linear_vel_min = 0.0
        self.linear_vel_max = 0.0
        self.angle_vel_min = 0.0
        self.angle_vel_max = 0.0

    def calc_Vr(self, vs: Vs, vd: Vd):
        self.linear_vel_min = max(vs.linear_vel_min, vd.linear_vel_min)
        self.linear_vel_max = min(vs.linear_vel_max, vd.linear_vel_max)
        self.angle_vel_min = max(vs.angle_vel_min, vd.angle_vel_min)
        self.angle_vel_max = min(vs.angle_vel_max, vd.angle_vel_max)

class DynamicWindowApproach:

    def __init__(self, x_init,
                       linear_vel_min, linear_vel_max, linear_acc_min, linear_acc_max,
                       angle_vel_min, angular_vel_max, angle_acc_min, angle_acc_max,
                       predict_time, dt,
                       linear_vel_resolution, angle_vel_resolution,
                       to_goal_cost_gain, speed_cost_gain, obstacle_cost_gain,
                       lidar_angle_min, lidar_angle_resolution,
                       robot_radius):
        self.x = x_init
        self.predict_time = predict_time
        self.dt = dt
        self.linear_vel_resolution = linear_vel_resolution
        self.angle_vel_resolution = angle_vel_resolution
        self.to_goal_cost_gain = to_goal_cost_gain
        self.speed_cost_gain = speed_cost_gain
        self.obstacle_cost_gain = obstacle_cost_gain
        self.lidar_angle_min = lidar_angle_min
        self.lidar_angle_resolution = lidar_angle_resolution
        self.robot_radius = robot_radius

        # Velocity space（ロボットが取りうる速度と角速度の範囲）
        self.vs = Vs(linear_vel_min, linear_vel_max, angle_vel_min, angular_vel_max)

        self.linear_acc_min = linear_acc_min
        self.linear_acc_max = linear_acc_max
        self.angle_acc_min = angle_acc_min
        self.angle_acc_max = angle_acc_max

    def motion_model(self, x, u, dt):
        """
        2次元平面の二輪作動ロボットの運動モデルにより、「現在の状態x」「入力u」から「次の状態x_next」を計算
        
        Parameters
        ----------
        x : numpy.ndarray
            現在の状態 [x, y, theta]
        u : numpy.ndarray
            入力 [v, w]
        dt : float
            時間ステップ

        Returns
        -------
        x_next : numpy.ndarray
            次の状態 [x, y, theta]
        """
        theta = x[2]
        F = np.array([[1.0, 0.0, 0.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0, 0.0, 0.0],
                      [0.0, 0.0, 1.0, 0.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0, 0.0]])
        B = np.array([[dt * np.cos(theta), 0.0],
                      [dt * np.sin(theta), 0.0],
                      [0.0, dt],
                      [1.0, 0.0],
                      [0.0, 1.0]])
        x_next = F @ x + B @ u # 摩擦の影響を考慮しない運動モデル
        return x_next


    def calc_best_u(self, x, u, lidar_results, goal):
        """
        最適な速度・角速度を計算

        Parameters
        ----------
        x : numpy.ndarray
            現在の状態 [x, y, theta]
        u : numpy.ndarray
            現在の速度・角速度 [v, w]
        lidar_results : list
            lidarの結果
        goal : numpy.ndarray
            ゴール位置 [x, y]

        Returns
        -------
        u_best : numpy.ndarray
            dynamic window approachによって算出された最適な速度・角速度 [v, w]
        """
        # Vrの計算
        vd = Vd()
        vd.calc_Vd(u[0], u[1], self.linear_acc_min, self.linear_acc_max, self.angle_acc_min, self.angle_acc_max, self.dt)
        vr = Vr()
        vr.calc_Vr(self.vs, vd)

        # 最適な速度・角速度の探索
        u_best = np.array([0.0, 0.0])
        min_cost = 1000000
        for linear_vel_candidate in np.arange(vr.linear_vel_min, vr.linear_vel_max, self.linear_vel_resolution):
            for angle_vel_candidate in np.arange(vr.angle_vel_min, vr.angle_vel_max, self.angle_vel_resolution):
                # 予測軌道の計算
                u_candidate = np.array([linear_vel_candidate, angle_vel_candidate])
                trajectory = self.calc_predict_trajectory(x, u_candidate, self.predict_time, self.dt)

                # コスト計算
                to_goal_cost = self.calc_to_goal_cost(trajectory, goal)
                speed_cost = self.calc_speed_cost(trajectory, self.vs)
                obstacle_cost = self.calc_obstacle_cost(trajectory, lidar_results, self.robot_radius)
                total_cost = self.to_goal_cost_gain * to_goal_cost + self.speed_cost_gain * speed_cost + self.obstacle_cost_gain * obstacle_cost
                
                # 最小コストの更新
                if total_cost < min_cost:
                    min_cost = total_cost
                    u_best = np.array([linear_vel_candidate, angle_vel_candidate])
        return u_best

    def calc_predict_trajectory(self, x, u, predict_time, dt):
        """
        現在の状態xと制御入力uから、predict_time後までの軌道を計算

        Parameters
        ----------
        x : numpy.ndarray
            現在の状態 [x, y, theta]
        u : numpy.ndarray
            制御入力 [v, w]
        dt : float
            時間ステップ

        Returns
        -------
        trajectory : numpy.ndarray
            軌道 [x, y, theta, v, w]
        """
        trajectory = np.array([x])
        current_time = 0.0
        while current_time <= predict_time:
            x = self.motion_model(x, u, dt)
            trajectory = np.vstack((trajectory, x))
            current_time += dt
        return trajectory            
    
    def calc_to_goal_cost(self, trajectory, goal):
        """
        ゴールまでのコストを計算
        軌道の最終位置とゴール位置の角度の差をコストとする（ゴールの方向に向かっているほどコストが低い）

        Parameters
        ----------
        trajectory : numpy.ndarray
            軌道 [x, y, theta, v, w]
        goal : numpy.ndarray
            ゴール位置 [x, y]

        Returns
        -------
        cost : float
            ゴールまでのコスト
        """
        dx = goal[0] - trajectory[-1, 0]
        dy = goal[1] - trajectory[-1, 1]
        error_angle = math.atan2(dy, dx)
        cost_angle = error_angle - trajectory[-1, 2]
        cost = abs(math.atan2(math.sin(cost_angle), math.cos(cost_angle)))
        return cost
    
    def calc_speed_cost(self, trajectory, vs: Vs):
        """
        速度に関するコストを計算
        現在の速度との差をコストとする（速度が速いほどコストが低い）

        Parameters
        ----------
        trajectory : numpy.ndarray
            軌道 [x, y, theta, v, w]
        vs : Vs
            velocity space

        Returns
        -------
        cost : float
            速度に関するコスト
        """
        cost = vs.linear_vel_max - trajectory[-1, 3]    
        return cost

    def calc_obstacle_cost(self, trajectory, lidar_results, robot_radius):
        """
        障害物に関するコストを計算
        軌道上の各点における障害物との距離の逆数をコストとする（障害物から遠いほどコストが低い）

        Parameters
        ----------
        trajectory : numpy.ndarray
            軌道 [x, y, theta, v, w]
        lidar_results : list

        Returns
        -------
        cost : float
            障害物に関するコスト
        """
        min_dist = 100000
        for trajectory_point_x, trajectory_point_y , _, _, _ in trajectory:
            for lidar_result in lidar_results:
                hit_position = lidar_result[3] # lidarの当たった位置（ワールド座標系）
                # lidarがhitしていない場合はスキップ（hitしていない場合、hit_point=[0, 0, 0]となる）
                if hit_position[0] == 0.0 and hit_position[1] == 0.0:
                    continue
                # 「予測軌道上の点」と「rayが当たった位置」との距離が近いほど、「予測軌道上の近い位置に障害物がある」と判断されるため、コストが高くなる
                dx = trajectory_point_x - hit_position[0]
                dy = trajectory_point_y - hit_position[1]
                dist = np.hypot(dx, dy) - robot_radius
                if dist < min_dist:
                    min_dist = dist
        cost = 1.0 / (min_dist + 1e-6)
        return cost



# <a id='toc5_'></a>[初期設定](#toc0_)

環境や、ロボットの生成などの処理設定を行います。

In [ ]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 0.1
pybullet.setTimeStep(time_step) # 1stepあたりに経過する時間の設定

# GUIモードの際のカメラの位置などを設定
camera_distance = 6.0
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# ロボットの読み込み
car_start_pos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
car_start_yaw = math.radians(0)
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, car_start_yaw]) # 初期姿勢(roll, pitch, yaw)をクォータニオンに変換
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf",car_start_pos, car_start_orientation)

# ゴール地点の球体
goal_visual_id = pybullet.createVisualShape(pybullet.GEOM_SPHERE, radius=0.1, rgbaColor=[0,1,0,1])
goal_id = pybullet.createMultiBody(0, -1, goal_visual_id, [0.0, 0.0, 0.0], useMaximalCoordinates=True)

ven = Mesa


# <a id='toc6_'></a>[移動ロボットのパラメータ設定](#toc0_)

In [5]:
# 左右の車輪のジョイントのインデックス
RIGHT_WHEEL_IDX = 0
LEFT_WHEEL_IDX = 1
WHEEL_THREAD = 0.325 # 車輪の間隔(「simple_two_wheel_car.urdf」の車輪の間隔と一致させる)

# <a id='toc7_'></a>[lidarのパラメータ設定](#toc0_)

In [6]:
LIDAR_LINK_IDX = 5 # rayの発射位置のlinkのインデックス
lidar_ray_origin_offset = 0.1 # lidarの原点からのオフセット[m]
lidar_ray_length = 2.0 # rayの長さ
lidar_angle_min = math.radians(-180) # rayの最小角度
lidar_angle_max = math.radians(180) # rayの最大角度
lidar_angle_resolution = math.radians(5) # rayの角度分解能

# <a id='toc8_'></a>[障害物設定](#toc0_)

障害物を生成します。

**障害物の位置によって、dynamic window approachの制御が変化するので、任意の位置に変更したり、追加したりしてみてください。**


In [7]:
# 障害物を追加
mass = 200 # kg
box_size = [0.5, 0.5, 0.3]
position = [2.0, 2.0, 0.3]
orientation = [1, 0, 0, 0] # 四元数
box_collision_id = pybullet.createCollisionShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physics_client)
box_visual_id = pybullet.createVisualShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physics_client, rgbaColor=[1,0,0,1]) # 赤・半透明
box_body_id = pybullet.createMultiBody(mass, box_collision_id, box_visual_id, position, orientation, physicsClientId=physics_client)

# <a id='toc9_'></a>[dynamic window approach パラメータ設定](#toc0_)

dynamic window approachに関連するパラメータを設定します。

**各パラメータを変更して、ロボットの動作を確認してみてください。**


---

注意：dynamic window approachは障害物の位置、目標位置などによっては、局所最適解に陥り、目標地点に到達できない場合があります。

そのため、冒頭に述べたように  
- グローバルパスプランニングで大域的な経路を求める
- → 求めた経路をいくつかの中間地点に分割
- → 「中間地点1に向かって走行」→「中間地点1に到達したら、中間地点2に向かって走行」→　... を目標地点に到達するまで繰り返す（中間地点に分割することで、局所最適解に陥る可能性を減らす）

といったような手順で実行することが一般的です。（本notebookでは、グローバルパスプランニングは行っていません。）

---

In [26]:
# dynamic windowに関するパラメータ
linear_vel_min = -0.5 # ロボットの最小速度[m/s]
linear_vel_max = 1.0 # ロボットの最大速度[m/s]
linear_acc_min = -0.5 # ロボットの最小加速度[m/s^2]
linear_acc_max = 0.5 # ロボットの最大加速度[m/s^2]
angle_vel_min = math.radians(-80) # ロボットの最小角速度[rad/s]
angle_vel_max = math.radians(80) # ロボットの最大角速度[rad/s]
angle_acc_min = math.radians(-40) # ロボットの最小角加速度[rad/s^2]
angle_acc_max = math.radians(40) # ロボットの最大角加速度[rad/s^2]
predict_time = 2.0 # 予測時間[s]（予測する軌跡を、何時刻先まで計算するか）
linear_vel_resolution = 0.02 # 速度の分解能[m/s]
angle_vel_resolution = 0.02 # 角速度の分解能[rad/s]
robot_radius = 0.1 # ロボットの半径[m]

# コストに関するパラメータ
to_goal_cost_gain = 0.15 # ゴールに向かうコストの重み（ゴールと違う方向を向いているほどコストが大きくなる）
speed_cost_gain = 1.0 # 速度に関するコストの重み（速度が遅いほどコストが大きくなる）
obstacle_cost_gain = 0.7 # 障害物に関するコストの重み（障害物に近づくほどコストが大きくなる）

# ゴール位置の設定
goal = np.array([1, 5, 0])
# goal = np.array([4, 4, 0])
# goal = np.array([3.5, 2, 0])

# ゴール位置を示す球体オブジェクトを、ゴール位置に配置
pybullet.resetBasePositionAndOrientation(goal_id, goal, [0, 0, 0, 1])

# <a id='toc10_'></a>[dynamic window approachの実行](#toc0_)

指定されたパラメータに基づいて、dynamic window approachを実行します。

In [28]:
import sys

# debug用の描画を初期化
pybullet.removeAllUserDebugItems()

# ロボットを初期位置にセット
pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)

# lidarの初期化
lidar = Lidar()

# dynamic window approachの初期化
dwa = DynamicWindowApproach(x_init=np.array(car_start_pos), 
                            linear_vel_min=linear_vel_min, linear_vel_max=linear_vel_max, linear_acc_min=linear_acc_min, linear_acc_max=linear_acc_max,
                            angle_vel_min=angle_vel_min, angular_vel_max=angle_vel_max, angle_acc_min=angle_acc_min, angle_acc_max=angle_acc_max,
                            predict_time=predict_time, dt=time_step,
                            linear_vel_resolution=linear_vel_resolution, angle_vel_resolution=angle_vel_resolution,
                            to_goal_cost_gain=to_goal_cost_gain, speed_cost_gain=speed_cost_gain, obstacle_cost_gain=obstacle_cost_gain,
                            lidar_angle_min=lidar_angle_min, lidar_angle_resolution=lidar_angle_resolution,
                            robot_radius=robot_radius)


# 初期制御入力
v_init = 0.0
omega_init = math.radians(0.0)
u = np.array([v_init, omega_init])

# 初期状態
x = np.array([car_start_pos[0], car_start_pos[1], car_start_yaw, v_init, omega_init])

while True:
    # lidar情報の取得
    lidar_position = pybullet.getLinkState(car_id, LIDAR_LINK_IDX) # lidar linkの情報を取得
    ray_from_position = [lidar_position[0][0], lidar_position[0][1], lidar_position[0][2]] # rayの始点をlidar linkの原点に設定
    lidar_orientation = list(pybullet.getEulerFromQuaternion(lidar_position[1])) 
    lidar_results = lidar.CheckHits(ray_from_position, lidar_orientation, lidar_ray_length,  lidar_angle_resolution, [lidar_angle_min, lidar_angle_max], lidar_ray_origin_offset, draw_debug_line=True)
    
    # dynamic window approachのメイン処理 ========================================
    u = dwa.calc_best_u(x, u, lidar_results, goal) # 最適な制御入力を計算
    # =========================================================================
    v = u[0]
    omega = u[1]
    sys.stdout.write(f"\r v: {round(v, 3)} omega: {round(omega, 3)}       ")

    # 並進速度、回転速度から左右の車輪の指令速度を計算
    right_wheel_velocity = v + omega * WHEEL_THREAD / 2
    left_wheel_velocity = v - omega * WHEEL_THREAD / 2

    # 速度を設定
    pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_wheel_velocity)
    pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_wheel_velocity)

    # シミュレーションを1ステップ進める
    pybullet.stepSimulation()

    # 状態の更新（ここで、「x, y, yaw」はシミュレーションから直接取得した真の値を使用。実際は、オドメトリなどを用いて推定された値を使用）
    car_position, car_orientation = pybullet.getBasePositionAndOrientation(car_id)
    car_euler = pybullet.getEulerFromQuaternion(car_orientation)
    car_yaw = car_euler[2]
    x = np.array([car_position[0], car_position[1], car_yaw, v, omega], dtype=np.float64)

    # ゴール付近に到達したら終了
    goal_dis = 0.3
    if np.linalg.norm(goal[:2] - x[:2]) < goal_dis:
        pybullet.addUserDebugText("GOAL!!", [goal[0]-0.7, goal[1]-0.8, 0.3], [0, 0, 1], textSize=2)
        break


 v: 0.13 omega: 0.154       